In [ ]:
require 'json'
require 'parallel'
require 'securerandom'


def run_cmd(ip,cmd)
  puts "====begin cmd #{time=Time.now} @#{ip}===="
  cmd = "ssh -i ~/.ssh/LightsailDefaultKey-us-east-1.pem -o 'StrictHostKeyChecking no' ubuntu@#{ip} '#{cmd}'"
  puts cmd
  system(cmd)
  puts "====end cmd #{Time.now} @#{ip} time #{Time.now-time} s===="
end


def worker_run(worker,public_ips,cmd)
  puts "====worker_run begin cmd #{time=Time.now}===="
  Parallel.map(worker,in_threads: 10) { |w| run_cmd(public_ips[w],cmd) }
  puts "====worker_run end cmd #{Time.now} time #{Time.now-time} s===="
end

def worker_run_script(worker,public_ips,script)
  cmd = script.split("\n")
  cmd.map {|c|
    worker_run(worker,public_ips,c)
  }
end

def get_public_ip
  get_public_ip_str = "aws lightsail get-instances --no-cli-pager --region 'us-east-1' --query 'instances[].{name:name,publicIpAddress:publicIpAddress}'"
  puts get_public_ip_str
  data = `#{get_public_ip_str}`
  public_ips = JSON.parse(data).map {|x| [x["name"],x["publicIpAddress"]]}.to_h
end

public_ips = get_public_ip




aws lightsail get-instances --no-cli-pager --region 'us-east-1' --query 'instances[].{name:name,publicIpAddress:publicIpAddress}'


JSON::ParserError: 783: unexpected token at ''

In [ ]:
dockers_per_instance = 6
worker = (3..3).map {|x| "panworker-#{x}"}

["panworker-3"]

In [ ]:
## start ec2
if worker.size>1 then
  create_worker = "aws lightsail create-instances --no-cli-pager --instance-names {#{worker.map{|x| "'#{x}'"}.join(',')}} --availability-zone 'us-east-1a' --blueprint-id 'ubuntu_20_04' --bundle-id 'medium_2_0'"
else
  create_worker = "aws lightsail create-instances --no-cli-pager --instance-names #{worker.map{|x| "'#{x}'"}.join(',')} --availability-zone 'us-east-1a' --blueprint-id 'ubuntu_20_04' --bundle-id 'medium_2_0'"
end
puts create_worker
system(create_worker)

public_ips = get_public_ip

## install docker
script1 = '''sudo apt-get update
sudo apt-get install -y ca-certificates curl gnupg lsb-release
curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo gpg --dearmor -o /usr/share/keyrings/docker-archive-keyring.gpg
echo "deb [arch=$(dpkg --print-architecture) signed-by=/usr/share/keyrings/docker-archive-keyring.gpg] https://download.docker.com/linux/ubuntu $(lsb_release -cs) stable" | sudo tee /etc/apt/sources.list.d/docker.list > /dev/null
sudo apt-get update
sudo apt-get install -y docker-ce docker-ce-cli containerd.io
sudo usermod -aG docker $USER'''

script2 = "newgrp docker"

script3 = """sudo systemctl enable docker.service
sudo systemctl enable containerd.service"""

worker_run_script(worker,public_ips,script1)

Parallel.map(worker,in_threads: 10) { |w| 
  ip = public_ips[w]
  cmd = "ssh -i ~/.ssh/LightsailDefaultKey-us-east-1.pem -o 'StrictHostKeyChecking no' ubuntu@#{ip} '#{script2}'<<EOF\nexit\nEOF\n"
  puts cmd
  system(cmd)
}

worker_run_script(worker,public_ips,script3)
worker_run_script(worker,public_ips,"docker run hello-world")

## start dockers
stop_script = """docker stop $(docker ps -a -q)
docker rm $(docker ps -a -q)"""

start_script = "docker container run -d --restart=always -e DB_CONNECT_STR='__PARAMS_CONNECT_STR__' -e WORKER_NAME='__WORKER__' --name __WORKER__  adam429/pan-repo:panworker"


worker_run_script(worker,public_ips,stop_script)

output = Parallel.map(worker,in_threads: 10) { |w| 

  script_a = start_script.gsub(/__WORKER__/,"#{w}_#{SecureRandom.hex(2)}").gsub(/__PARAMS_CONNECT_STR__/,+ENV["DB_CONNECT_STR"])
  worker_run_script([w],public_ips,script_a)

  Parallel.map((2..dockers_per_instance).to_a, in_threads: 10) { |i|
    script_a = start_script.gsub(/__WORKER__/,"#{w}_#{SecureRandom.hex(2)}").gsub(/__PARAMS_CONNECT_STR__/,+ENV["DB_CONNECT_STR"])
    worker_run_script([w],public_ips,script_a)
  }
}



In [ ]:
worker_run_script(worker,public_ips,"docker run hello-world")


====worker_run begin cmd 2022-03-02 16:51:54 +0000====


NoMethodError: undefined method `[]' for nil:NilClass

In [ ]:
# delete workers

Parallel.map(worker,in_threads: 10) { |w| 
  delete_worker = "aws lightsail delete-instance --no-cli-pager --instance-name #{w}" 
  puts delete_worker
  system(delete_worker)
}
public_ips = get_public_ip


In [ ]:
runner = "panworker-0_6781"

instance, _ = runner.split("_")
docker = runner

        get_public_ip_str = "aws lightsail get-instances --no-cli-pager --region 'us-east-1' --query 'instances[].{name:name,publicIpAddress:publicIpAddress}'"
        data = `#{get_public_ip_str}`
        public_ips = JSON.parse(data).map {|x| [x["name"],x["publicIpAddress"]]}.to_h
        ip = public_ips[instance]
        cmd = "docker restart #{docker}"          
        `ssh -i ~/.ssh/LightsailDefaultKey-us-east-1.pem -o 'StrictHostKeyChecking no' ubuntu@#{ip} '#{cmd}'`


JSON::ParserError: 783: unexpected token at ''

In [ ]:
script = """sudo reboot now"""

worker_run_script(worker,public_ips,script)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ebde59c2-d67b-4e31-ad3b-1bdd07014691' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>